# 음성합성 데모

이 문서는 T2T, TTS 전과정을 거쳐 표준어 텍스트로 경상도 사투리 음성 합성을 진행하는 데모입니다.

# T2T
표준어 텍스트를 경상도 사투리 텍스트로 변환합니다.

In [ ]:
! pip install kss transformers

In [ ]:
import kss
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
def standard_text_to_dialect_text(prompt):
    with torch.no_grad():
        splited_sentences = kss.split_sentences(prompt)
        generated = []
        for sentence in splited_sentences:
            tokens = tokenizer.encode(sentence, return_tensors="pt").to(
                device="cuda", non_blocking=True
            )
            gen_tokens = model.generate(
                tokens, max_length=256, repetition_penalty=1.5
            )
            generated.append(
                tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)[0]
            )
    return ' '.join(generated)

In [ ]:
T2T_MODEL_NAME = "dannykm/DialectKoUL2"

tokenizer = AutoTokenizer.from_pretrained(T2T_MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(T2T_MODEL_NAME).to(device="cuda")
_ = model.eval()

prompt = input("텍스트를 입력하세요: ")
dialect_text = standard_text_to_dialect_text(prompt)
print(f"사투리로 변환된 텍스트: {dialect_text}")

# TTS

### 1. 필수 라이브러리 및 함수 불러오기

실행에 필요한 라이브러리 및 함수를 불러옵니다.

이 과정은 약 10분 정도 소요될 수 있습니다.

In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
!git clone --depth 1 https://github.com/sce-tts/g2pK.git
%cd /content/TTS
!pip install -q --no-cache-dir -e .
%cd /content/g2pK
!pip install -q --no-cache-dir "konlpy" "jamo" "nltk" "python-mecab-ko"
!pip install -q --no-cache-dir -e .

In [ ]:
%cd /content/g2pK
import g2pk
g2p = g2pk.G2p()

In [ ]:
!pip install pysbd coqpit unidecode pypinyin librosa==0.9.1

In [ ]:
%cd /content/TTS
import re
import sys
from unicodedata import normalize
import IPython

from TTS.utils.synthesizer import Synthesizer

def normalize_text(text):
    text = text.strip()

    for c in ",;:":
        text = text.replace(c, ".")
    text = remove_duplicated_punctuations(text)

    text = jamo_text(text)

    text = g2p.idioms(text)
    text = g2pk.english.convert_eng(text, g2p.cmu)
    text = g2pk.utils.annotate(text, g2p.mecab)
    text = g2pk.numerals.convert_num(text)
    text = re.sub("/[PJEB]", "", text)

    text = alphabet_text(text)

    # remove unreadable characters
    text = normalize("NFD", text)
    text = "".join(c for c in text if c in symbols)
    text = normalize("NFC", text)

    text = text.strip()
    if len(text) == 0:
        return ""

    # only single punctuation
    if text in '.!?':
        return punctuation_text(text)

    # append punctuation if there is no punctuation at the end of the text
    if text[-1] not in '.!?':
        text += '.'

    return text


def remove_duplicated_punctuations(text):
    text = re.sub(r"[.?!]+\?", "?", text)
    text = re.sub(r"[.?!]+!", "!", text)
    text = re.sub(r"[.?!]+\.", ".", text)
    return text


def split_text(text):
    text = remove_duplicated_punctuations(text)

    texts = []
    for subtext in re.findall(r'[^.!?\n]*[.!?\n]', text):
        texts.append(subtext.strip())

    return texts


def alphabet_text(text):
    text = re.sub(r"(a|A)", "에이", text)
    text = re.sub(r"(b|B)", "비", text)
    text = re.sub(r"(c|C)", "씨", text)
    text = re.sub(r"(d|D)", "디", text)
    text = re.sub(r"(e|E)", "이", text)
    text = re.sub(r"(f|F)", "에프", text)
    text = re.sub(r"(g|G)", "쥐", text)
    text = re.sub(r"(h|H)", "에이치", text)
    text = re.sub(r"(i|I)", "아이", text)
    text = re.sub(r"(j|J)", "제이", text)
    text = re.sub(r"(k|K)", "케이", text)
    text = re.sub(r"(l|L)", "엘", text)
    text = re.sub(r"(m|M)", "엠", text)
    text = re.sub(r"(n|N)", "엔", text)
    text = re.sub(r"(o|O)", "오", text)
    text = re.sub(r"(p|P)", "피", text)
    text = re.sub(r"(q|Q)", "큐", text)
    text = re.sub(r"(r|R)", "알", text)
    text = re.sub(r"(s|S)", "에스", text)
    text = re.sub(r"(t|T)", "티", text)
    text = re.sub(r"(u|U)", "유", text)
    text = re.sub(r"(v|V)", "브이", text)
    text = re.sub(r"(w|W)", "더블유", text)
    text = re.sub(r"(x|X)", "엑스", text)
    text = re.sub(r"(y|Y)", "와이", text)
    text = re.sub(r"(z|Z)", "지", text)

    return text


def punctuation_text(text):
    # 문장부호
    text = re.sub(r"!", "느낌표", text)
    text = re.sub(r"\?", "물음표", text)
    text = re.sub(r"\.", "마침표", text)

    return text


def jamo_text(text):
    # 기본 자모음
    text = re.sub(r"ㄱ", "기역", text)
    text = re.sub(r"ㄴ", "니은", text)
    text = re.sub(r"ㄷ", "디귿", text)
    text = re.sub(r"ㄹ", "리을", text)
    text = re.sub(r"ㅁ", "미음", text)
    text = re.sub(r"ㅂ", "비읍", text)
    text = re.sub(r"ㅅ", "시옷", text)
    text = re.sub(r"ㅇ", "이응", text)
    text = re.sub(r"ㅈ", "지읒", text)
    text = re.sub(r"ㅊ", "치읓", text)
    text = re.sub(r"ㅋ", "키읔", text)
    text = re.sub(r"ㅌ", "티읕", text)
    text = re.sub(r"ㅍ", "피읖", text)
    text = re.sub(r"ㅎ", "히읗", text)
    text = re.sub(r"ㄲ", "쌍기역", text)
    text = re.sub(r"ㄸ", "쌍디귿", text)
    text = re.sub(r"ㅃ", "쌍비읍", text)
    text = re.sub(r"ㅆ", "쌍시옷", text)
    text = re.sub(r"ㅉ", "쌍지읒", text)
    text = re.sub(r"ㄳ", "기역시옷", text)
    text = re.sub(r"ㄵ", "니은지읒", text)
    text = re.sub(r"ㄶ", "니은히읗", text)
    text = re.sub(r"ㄺ", "리을기역", text)
    text = re.sub(r"ㄻ", "리을미음", text)
    text = re.sub(r"ㄼ", "리을비읍", text)
    text = re.sub(r"ㄽ", "리을시옷", text)
    text = re.sub(r"ㄾ", "리을티읕", text)
    text = re.sub(r"ㄿ", "리을피읍", text)
    text = re.sub(r"ㅀ", "리을히읗", text)
    text = re.sub(r"ㅄ", "비읍시옷", text)
    text = re.sub(r"ㅏ", "아", text)
    text = re.sub(r"ㅑ", "야", text)
    text = re.sub(r"ㅓ", "어", text)
    text = re.sub(r"ㅕ", "여", text)
    text = re.sub(r"ㅗ", "오", text)
    text = re.sub(r"ㅛ", "요", text)
    text = re.sub(r"ㅜ", "우", text)
    text = re.sub(r"ㅠ", "유", text)
    text = re.sub(r"ㅡ", "으", text)
    text = re.sub(r"ㅣ", "이", text)
    text = re.sub(r"ㅐ", "애", text)
    text = re.sub(r"ㅒ", "얘", text)
    text = re.sub(r"ㅔ", "에", text)
    text = re.sub(r"ㅖ", "예", text)
    text = re.sub(r"ㅘ", "와", text)
    text = re.sub(r"ㅙ", "왜", text)
    text = re.sub(r"ㅚ", "외", text)
    text = re.sub(r"ㅝ", "워", text)
    text = re.sub(r"ㅞ", "웨", text)
    text = re.sub(r"ㅟ", "위", text)
    text = re.sub(r"ㅢ", "의", text)

    return text


def normalize_multiline_text(long_text):
    texts = split_text(long_text)
    normalized_texts = [normalize_text(text).strip() for text in texts]
    return [text for text in normalized_texts if len(text) > 0]

def synthesize(text):
    wavs = synthesizer.tts(text, None, None)
    return wavs

### 2. 학습한 모델 불러오기

Hugging face에 저장해둔 학습시킨 Glow-TTS와 HiFi-GAN 모델을 불러옵니다.
- https://huggingface.co/soohyunn/glow-tts
- https://huggingface.co/soohyunn/hifi-GAN

<br>
만약 다른 체크포인트에서 불러오시려면 아래 코드에서 경로를 아래와 같이 적절하게 수정합니다.

```python
synthesizer = Synthesizer(
    "/content/data/glow-tts/glowtts-v2-December-04-2023_10+51AM-3aa165a/best_model.pth.tar",
    "/content/data/glow-tts/glowtts-v2-December-04-2023_10+51AM-3aa165a/config.json",
    None,
    "/content/data/hifi-GAN/hifigan-v2-December-04-2023_01+59PM-3aa165a/checkpoint_300000.pth.tar",
    "/content/data/hifi-GAN/hifigan-v2-December-04-2023_01+59PM-3aa165a/config.json",
    None,
    None,
    False,
)
```

In [ ]:
# Hugging face 에서 학습시킨 모델 불러오기
import os

os.makedirs("/content/data")
os.chdir("/content/data")
!git clone https://huggingface.co/soohyunn/glow-tts
!git clone https://huggingface.co/soohyunn/hifi-GAN

In [ ]:
import json
stat_path = ['/content/data/glow-tts/scale_stats_new.npy', '/content/data/hifi-GAN/scale_stats_new.npy']
i = 0

for file_path in roots:
  with open(file_path, 'r') as file:
      data = json.load(file)

  data["audio"]["stats_path"] = stat_path[i]

  with open(file_path, 'w', encoding='utf-8') as file:
      json.dump(data, file, indent="\t")
  i+=1

In [ ]:
# 체크포인트에서 모델 불러와서 음성합성 준비
synthesizer = Synthesizer(
    "/content/data/glow-tts/glowtts-v2-December-04-2023_10+51AM-3aa165a/checkpoint_70000.pth.tar",
    "/content/data/glow-tts/glowtts-v2-December-04-2023_10+51AM-3aa165a/config.json",
    None,
    "/content/data/hifi-GAN/hifigan-v2-December-04-2023_01+59PM-3aa165a/checkpoint_400000.pth.tar",
    "/content/data/hifi-GAN/hifigan-v2-December-04-2023_01+59PM-3aa165a/config.json",
    None,
    None,
    False,
)
symbols = synthesizer.tts_config.characters.characters

### 3. 음성 합성

여기서는 실제 음성 합성을 수행합니다.

위의 T2T를 통해 나온 경상도 방언 텍스트 `dialect_text`를 TTS의 input text로 사용합니다.

In [ ]:
for text in normalize_multiline_text(dialect_text):
    wav = synthesizer.tts(text, None, None)
    IPython.display.display(IPython.display.Audio(wav, rate=22050))